In [2]:
import numpy as np
import astropy.units as u
from astropy.coordinates import EarthLocation


In [98]:
(7.7 * u.cm * 7500).to(u.m)

<Quantity 577.5 m>

In [3]:
# canguro: 12.500438, 46.030509
# incrocio: 12.508266, 46.026821
# parcheggio: 12.499188, 46.019864

canguro = EarthLocation.from_geodetic(12.500438, 46.030509)

inizio_sentiero = EarthLocation.from_geodetic(12.500892, 46.029589)

cima_collina = EarthLocation.from_geodetic(12.500701, 46.025092)

parcheggio = EarthLocation.from_geodetic(12.499188, 46.019864)

def p(pos):
    return np.array([pos.x.value, pos.y.value, pos.z.value], dtype=np.float64) * pos.unit

In [32]:
def az_dist(p1, p2):
    # definitely broken, do not use
    sep_vector = p(p2) - p(p1)
    radius_earth = p(p1)
    projector_planar = np.eye(3) - np.tensordot(radius_earth, radius_earth, axes=0) / np.linalg.norm(radius_earth)**2
    
    dist = np.linalg.norm(sep_vector)
    north = projector_planar @ np.array([0, 0, 1])
    # east = projector_planar@ (np.cross(radius_earth, north) / np.linalg.norm(radius_earth))
    az = np.arccos(north @ sep_vector / dist).to(u.degree)
    print(sep_vector / dist)
    print(north)
    # if sep_vector@east > 0: 
    #     az+= 180 * u.degree
    return(az, dist)

In [27]:
print(az_dist(canguro, inizio_sentiero))
print(az_dist(incrocio, parcheggio))

(<Quantity 131.21209595 deg>, <Quantity 108.13200296 m>)


NameError: name 'incrocio' is not defined

In [28]:
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')

def az_dist_2(p1, p2):
    lat1, long1 = p1.lat.value, p1.lon.value
    lat2, long2 = p2.lat.value, p2.lon.value
    fwd_azimuth, back_azimuth, distance = geodesic.inv(lat1, long1, lat2, long2)
    return(270-back_azimuth, distance)

In [29]:
az_dist_2(canguro, inizio_sentiero)

(153.33270675810263, 111.90546380654924)

In [30]:
test = EarthLocation.from_geodetic(0, 45)
test_north = EarthLocation.from_geodetic(0, 45+1e-4)
test_east = EarthLocation.from_geodetic(1e-4, 45)

In [33]:
print(az_dist(test, test_north))
print(az_dist(test, test_east))
print(az_dist_2(test, test_north))
print(az_dist_2(test, test_east))

[-0.7071074   0.          0.70710616]
[-0.49998872  0.          0.50335839]
(<Quantity 44.80790225 deg>, <Quantity 11.11317784 m>)
[-8.72655342e-07  1.00000000e+00  0.00000000e+00]
[-0.49998872  0.          0.50335839]
(<Quantity 89.999975 deg>, <Quantity 7.88468351 m>)
(360.0, 11.131949081278844)
(90.0, 11.057427582159868)


In [22]:
aa.R_earth * np.cos(45 * u.degree) * 1e-4 * np.pi / 180

<Quantity 7.87143102 m>